In [3]:
import pandas as pd
import gensim
import os
import numpy as np

In [56]:
import os
import spacy
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

nlp = spacy.load("en_core_web_sm")

folder_path = r'C:\devdev\Games\GOT'

def remove_stopwords(tokens):
    return [word for word in tokens if word not in STOPWORDS]

def process_chunk(text_chunk): 
    """Tokenize sentences, preprocess words, and remove stopwords."""
    doc = nlp(text_chunk)
    for sent in doc.sents:
        tokens = simple_preprocess(sent.text)  # Tokenize sentence
        filtered_tokens = remove_stopwords(tokens)  # Remove stopwords
        story.append(filtered_tokens)

story = []

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)

    if file.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            while True:
                chunk = f.read(500_000)  # Read 500,000 characters at a time
                if not chunk:
                    break
                process_chunk(chunk)  # Process each chunk separately

print(story[:5]) 



[['game', 'thrones', 'book', 'song', 'ice', 'george', 'martin', 'prologue', 'start', 'gared', 'urged', 'woods', 'began', 'grow', 'dark'], ['wildlings', 'dead'], ['dead', 'frighten'], ['ser', 'waymar', 'royce', 'asked', 'hint', 'smile'], ['gared', 'rise', 'bait']]


In [57]:
len(story)

155037

In [74]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=story, vector_size=100, window=5, min_count=3, workers=4)
model.save("word2vec_GOT.model")

In [75]:
model = Word2Vec.load("word2vec_GOT.model")  

In [76]:
for word, vocab_obj in model.wv.key_to_index.items():
    print(f"{word}: {model.wv.get_vecattr(word, 'count')} times")

said: 8747 times
lord: 7816 times
ser: 6019 times
man: 4722 times
men: 3856 times
king: 3554 times
like: 3203 times
jon: 3139 times
old: 2787 times
hand: 2773 times
tyrion: 2669 times
know: 2653 times
father: 2598 times
eyes: 2530 times
black: 2511 times
told: 2510 times
thought: 2434 times
lady: 2413 times
time: 2341 times
long: 2307 times
come: 2251 times
face: 2139 times
head: 2093 times
red: 2083 times
way: 2067 times
page: 2022 times
boy: 1995 times
good: 1960 times
little: 1897 times
brother: 1877 times
took: 1875 times
came: 1857 times
night: 1787 times
away: 1761 times
queen: 1751 times
dead: 1746 times
son: 1695 times
blood: 1694 times
half: 1681 times
arya: 1630 times
saw: 1610 times
white: 1586 times
day: 1574 times
jaime: 1533 times
look: 1521 times
want: 1519 times
tell: 1477 times
sword: 1476 times
great: 1461 times
looked: 1454 times
ll: 1451 times
bran: 1451 times
girl: 1448 times
left: 1438 times
knew: 1436 times
gave: 1432 times
asked: 1432 times
called: 1428 times
wa

In [79]:
model.wv.most_similar("raze")

[('habitation', 0.9104482531547546),
 ('ends', 0.9088729023933411),
 ('beasts', 0.9073642492294312),
 ('fishing', 0.9053751826286316),
 ('lockstep', 0.9052647948265076),
 ('build', 0.9051240086555481),
 ('villages', 0.9048757553100586),
 ('rats', 0.9043810367584229),
 ('mounts', 0.9020062685012817),
 ('scatter', 0.9015456438064575)]

trying to get a heading for each topic

In [102]:
import spacy
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = nlp.Defaults.stop_words
print(sorted(spacy_stopwords))

["'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'he

In [105]:
import os
import numpy as np
import spacy
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS  # Import stopwords from spaCy

model = Word2Vec.load("word2vec_GOT.model")
nlp = spacy.load("en_core_web_sm")

CUSTOM_STOPWORDS = STOP_WORDS.union(nlp.Defaults.stop_words)

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in CUSTOM_STOPWORDS]

def get_paragraph_vector(paragraph, model):
    words = simple_preprocess(paragraph)  # Tokenize & preprocess
    clean_words = remove_stopwords(words)  # Remove stopwords
    word_vectors = [model.wv[word] for word in clean_words if word in model.wv]

    if word_vectors:
        return np.mean(word_vectors, axis=0)  # Average word embeddings
    else:
        return np.zeros(model.vector_size)  # Return zero vector if no words found

folder_path = r"GOT"

paragraph_vectors = []
paragraphs = []
file_names = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)

    if file_name.endswith('.txt'):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            while True:
                chunk = f.read(500_000)  # Read in chunks
                if not chunk:
                    break
                
                doc = nlp(chunk)  # Tokenize sentences
                for para in doc.sents:
                    para_text = para.text.strip()
                    
                    words = simple_preprocess(para_text)
                    clean_words = remove_stopwords(words)
                    clean_para = " ".join(clean_words)

                    if len(clean_para) > 30:  # Ignore very short sentences
                        paragraphs.append(clean_para)  # Store cleaned paragraph
                        file_names.append(file_name)
                        paragraph_vectors.append(get_paragraph_vector(clean_para, model))

# Convert to numpy array for clustering
paragraph_vectors = np.array(paragraph_vectors)

In [108]:
from sklearn.cluster import KMeans

num_cluster=5

kmeans= KMeans(n_clusters=num_cluster, random_state=42)
cluster_labels= kmeans.fit_predict(paragraph_vectors)

In [109]:
# Organize paragraphs into clusters
clustered_paragraphs = {}
for cluster_id in range(num_cluster):
    clustered_paragraphs[cluster_id] = []

In [110]:
# Assign each paragraph to its cluster
for index, (file, para) in enumerate(zip(file_names, paragraphs)):
    cluster_id = cluster_labels[index]
    clustered_paragraphs[cluster_id].append((file, para))

In [111]:
for cluster_id, paras in clustered_paragraphs.items():
    print(f"\n🔹 **Cluster {cluster_id}** (Sample Paragraphs):")
    
    for file, para in paras[:3]:  
        print(f"📌 [{file}] {para[:100]}...\n")  


🔹 **Cluster 0** (Sample Paragraphs):
📌 [001ssb.txt] known drag quarrel sooner later...

📌 [001ssb.txt] mother told dead men sing songs...

📌 [001ssb.txt] wet nurse said thing royce replied...


🔹 **Cluster 1** (Sample Paragraphs):
📌 [001ssb.txt] voice echoed loud twilit forest...

📌 [001ssb.txt] taste nervous tension came perilous close fear...

📌 [001ssb.txt] day felt watching cold implacable loved...


🔹 **Cluster 2** (Sample Paragraphs):
📌 [001ssb.txt] ser waymar royce asked hint smile...

📌 [001ssb.txt] ser waymar royce glanced sky disinterest...

📌 [001ssb.txt] ser waymar royce youngest son ancient house heirs...


🔹 **Cluster 3** (Sample Paragraphs):
📌 [001ssb.txt] unmanned dark gared tightness gared mouth barely sup pressed anger eyes thick black hood cloak...

📌 [001ssb.txt] cold wind blowing north trees rustle like living things...

📌 [001ssb.txt] handsome youth eighteen grey eyed graceful slender knife...


🔹 **Cluster 4** (Sample Paragraphs):
📌 [001ssb.txt] game thrones boo

In [ ]:
from collections import Counter
import spacy

nlp = spacy.load("en_core_web_sm")

def get_top_words(cluster_paragraphs, top_n=10):
    word_counts = Counter()
    for _, para in cluster_paragraphs:
        word_counts.update(para.split())  
    return [word for word, _ in word_counts.most_common(top_n)]

def get_named_entities(cluster_paragraphs, top_n=5):
    """Extracts top named entities (characters, places, etc.) for cluster naming."""
    entity_counts = Counter()
    for _, para in cluster_paragraphs:
        doc = nlp(para)
        entity_counts.update([ent.text for ent in doc.ents])  
    return [entity for entity, _ in entity_counts.most_common(top_n)]

cluster_names = {}
for cluster_id, paras in clustered_paragraphs.items():
    top_words = get_top_words(paras)
    top_entities = get_named_entities(paras)

    topic_keywords = ", ".join(top_words[:3])
    entity_keywords = ", ".join(top_entities[:2])  

    if entity_keywords:
        cluster_names[cluster_id] = f"Topic: {entity_keywords} | Keywords: {topic_keywords}"
    else:
        cluster_names[cluster_id] = f"Topic: {topic_keywords}"

for cluster_id, name in cluster_names.items():
    print(f"Cluster {cluster_id} → {name}")

Cluster 0 → Topic: half, seven | Keywords: said, man, told
Cluster 1 → Topic: half, brienne | Keywords: hand, man, head
Cluster 2 → Topic: stannis, seven | Keywords: lord, ser, king
Cluster 3 → Topic: half, morning | Keywords: black, red, eyes
Cluster 4 → Topic: half, seven | Keywords: men, great, night
